# Student Mental Health Analysis using SQL

This notebook analyzes student mental health data using SQL queries on a PostgreSQL database. We'll explore the relationship between international students' length of stay and their mental health indicators including depression, social connectedness, and acculturative stress.

## Dataset Overview
The dataset contains information about international and domestic students, including:
- Mental health scores (depression, social connectedness, acculturative stress)
- Demographic information (age, gender, academic level)
- Language proficiency and length of stay data

## Analysis Objectives:
- Load the Dataset
- Explore Data Structure
- Handle Missing Values
- Summary Statistics
- Visualize Categorical Distributions
- Visualize Numeric Distributions
- Correlation Analysis
- Group Analysis by Region and Academic Level

## 1. Database Connection Setup

We establish a connection to PostgreSQL using environment variables for security. This approach keeps sensitive database credentials out of the code and allows for easy configuration across different environments.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# PostgreSQL connection parameters
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')
db_host = os.getenv('POSTGRES_HOST')
db_port = os.getenv('POSTGRES_PORT')
db_name = os.getenv('POSTGRES_DB')


In [2]:
try:
    #create a connection string
    connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
    # Create a database engine
    engine = create_engine(connection_string)

    # Load the data into the database
    df = pd.read_csv('Data\students.csv')
    df.to_sql('students', engine, if_exists='replace', index=False)

    # Verify the data was loaded
    print("Successfully loaded data into the database.")

except Exception as e:
    print(f"An error occurred: {e}")

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\flega\AppData\Local\Temp\ipykernel_41764\2167192782.py:8: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('Data\students.csv')


Successfully loaded data into the database.


## 2. Data Loading and Verification

Load the student mental health dataset into PostgreSQL and verify successful import. We use `if_exists='replace'` to ensure a clean dataset and `index=False` to prevent pandas from adding an unnecessary index column.

In [3]:
#function to run sql query
def run_query(query):
    try:
        return pd.read_sql_query(query, engine)
    except Exception as e:
        print(f"An error occurred while executing the query: {e}")



## 3. Query Helper Function

Create a reusable function to execute SQL queries and handle errors gracefully. This function will be used throughout the analysis to run our PostgreSQL queries.

  | Field Name    | Description                                     |
  |---------------|-------------------------------------------------|
  | inter_dom     | Types of students (international or domestic)   |
  | japanese_cate | Japanese language proficiency                   |
  | english_cate  | English language proficiency                    |
  | academic      | Current academic level (undergraduate or graduate) |
  | age           | Current age of student                          |
  | stay          | Current length of stay in years                 |
  | todep         | Total score of depression (PHQ-9 test)          |
  | tosc          | Total score of social connectedness (SCS test)  |
  | toas          | Total score of acculturative stress (ASISS test)|

## 4. Data Dictionary

Understanding our key variables for mental health analysis:

In [4]:
#run test query
query = """
SELECT * FROM students LIMIT 5;
"""

run_query(query)

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,...,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,...,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,...,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,...,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,...,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No


## 5. Exploratory Data Analysis

### 5.1 Initial Data Exploration
First, let's examine the structure and content of our dataset to understand what we're working with. This gives us a quick overview of the data format and sample records.

In [5]:
#explore the students table structure
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'students';
"""

run_query(query)

,column_name,data_type
0,amiscell,double precision
1,toas,double precision
2,partner,double precision
3,friends,double precision
4,parents,double precision
5,relative,double precision
6,profess,double precision
7,phone,double precision
8,doctor,double precision
9,reli,double precision


### 5.2 Table Structure Analysis
Understanding the data types and column structure helps us write appropriate SQL queries and identify any data type issues that might affect our analysis.

In [6]:
#check how many rows are there in the students table
query = """
SELECT COUNT(*) FROM students;
"""

run_query(query)

,count
0,286


### 5.3 Dataset Size and Student Types
Let's check the total number of records and identify the types of students in our dataset. This helps us understand the scope of our data and plan appropriate analyses.

In [7]:
#check if students are international
query = """
SELECT DISTINCT inter_dom FROM students;
"""
run_query(query)

,inter_dom
0,None
1,Dom
2,Inter


In [8]:
#select the columns that we need for analysis
query = """
SELECT stay, todep, tosc, toas
FROM students;
"""

run_query(query)

,stay,todep,tosc,toas
0,5.0,0.0,34.0,91.0
1,1.0,2.0,48.0,39.0
2,6.0,2.0,41.0,51.0
3,1.0,3.0,37.0,75.0
4,1.0,3.0,37.0,82.0
...,...,...,...,...
281,NaN,NaN,NaN,NaN
282,NaN,NaN,NaN,NaN
283,NaN,NaN,NaN,NaN
284,NaN,NaN,NaN,NaN


### 5.4 Key Variables for Mental Health Analysis
Extracting the core variables we'll use for our mental health analysis. These four variables form the foundation of our study:
- **stay**: Length of stay in years (predictor variable)
- **todep**: Depression score from PHQ-9 test (outcome variable)
- **tosc**: Social connectedness score from SCS test (outcome variable) 
- **toas**: Acculturative stress score from ASISS test (outcome variable)

## 6. Mental Health Analysis by Length of Stay

### 6.1 International Students' Mental Health Metrics
This analysis focuses on international students and examines how their mental health indicators vary by length of stay. We calculate aggregate statistics for each stay duration:

- **Count**: Number of international students per stay duration
- **Average PHQ-9**: Depression levels (higher scores indicate more severe depression symptoms)
- **Average SCS**: Social connectedness (higher scores indicate better social connection and support)
- **Average ASISS**: Acculturative stress (higher scores indicate more stress from cultural adaptation)

**Interpretation Guide:**
- PHQ-9 scores: 0-4 (minimal), 5-9 (mild), 10-14 (moderate), 15-19 (moderately severe), 20-27 (severe)
- Higher SCS scores = Better social connectedness
- Higher ASISS scores = More acculturative stress

In [9]:
#create a count_int column for the number of international students for each length of stay
query = """
SELECT stay, COUNT(*) AS count_int, 
       ROUND(CAST(AVG(todep) AS numeric), 2) AS average_phq, 
       ROUND(CAST(AVG(tosc) AS numeric), 2) AS average_scs, 
       ROUND(CAST(AVG(toas) AS numeric), 2) AS average_as
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;
"""
run_query(query)

,stay,count_int,average_phq,average_scs,average_as
0,10.0,1,13.00,32.00,50.00
1,8.0,1,10.00,44.00,65.00
2,7.0,1,4.00,48.00,45.00
3,6.0,3,6.00,38.00,58.67
4,5.0,1,0.00,34.00,91.00
5,4.0,14,8.57,33.93,87.71
6,3.0,46,9.09,37.13,78.00
7,2.0,39,8.28,37.08,77.67
8,1.0,95,7.48,38.11,72.80


## 7. Language Proficiency and Mental Health Analysis

### 7.1 English Proficiency Impact on Depression
Let's examine whether English language proficiency affects depression levels among international students. Better language skills might correlate with lower depression scores.

In [10]:
# Analyze English proficiency levels among international students
query = """
SELECT english_cate, COUNT(*) AS student_count,
       ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
       ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness,
       ROUND(CAST(AVG(toas) AS numeric), 2) AS avg_acculturative_stress
FROM students
WHERE inter_dom = 'Inter' AND english_cate IS NOT NULL
GROUP BY english_cate
ORDER BY english_cate;
"""
run_query(query)

,english_cate,student_count,avg_depression,avg_social_connectedness,avg_acculturative_stress
0,Average,41,8.46,36.93,75.66
1,High,153,7.93,37.42,75.50
2,Low,7,8.00,40.29,76.43


### 7.2 Japanese Proficiency and Mental Health
Since this appears to be data from Japan, let's examine how Japanese language proficiency correlates with mental health outcomes among international students.

In [11]:
# Analyze Japanese proficiency levels among international students
query = """
SELECT japanese_cate, COUNT(*) AS student_count,
       ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
       ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness,
       ROUND(CAST(AVG(toas) AS numeric), 2) AS avg_acculturative_stress
FROM students
WHERE inter_dom = 'Inter' AND japanese_cate IS NOT NULL
GROUP BY japanese_cate
ORDER BY japanese_cate;
"""
run_query(query)

,japanese_cate,student_count,avg_depression,avg_social_connectedness,avg_acculturative_stress
0,Average,85,8.38,37.56,75.45
1,High,25,7.40,38.48,72.00
2,Low,91,7.91,36.99,76.65


## 8. Academic Level and Mental Health Analysis

### 8.1 Undergraduate vs Graduate Student Mental Health
Let's compare mental health outcomes between undergraduate and graduate international students to understand if academic level influences psychological well-being.

In [12]:
# Compare mental health by academic level
query = """
SELECT academic, COUNT(*) AS student_count,
       ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
       ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness,
       ROUND(CAST(AVG(toas) AS numeric), 2) AS avg_acculturative_stress,
       ROUND(CAST(AVG(age) AS numeric), 1) AS avg_age
FROM students
WHERE inter_dom = 'Inter' AND academic IS NOT NULL
GROUP BY academic
ORDER BY academic;
"""
run_query(query)

,academic,student_count,avg_depression,avg_social_connectedness,avg_acculturative_stress,avg_age
0,Grad,20,4.95,40.90,75.80,27.7
1,Under,181,8.39,37.03,75.54,20.3


## 9. Combined Analysis: Language Proficiency and Length of Stay

### 9.1 English Proficiency by Length of Stay
This analysis examines whether students with longer stays tend to have better English proficiency and how this relates to their mental health.

In [13]:
# Analyze relationship between stay duration and English proficiency
query = """
SELECT stay, english_cate, COUNT(*) AS student_count,
       ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression
FROM students
WHERE inter_dom = 'Inter' AND english_cate IS NOT NULL AND stay IS NOT NULL
GROUP BY stay, english_cate
ORDER BY stay, english_cate;
"""
run_query(query)

,stay,english_cate,student_count,avg_depression
0,1.0,Average,27,8.41
1,1.0,High,64,7.14
2,1.0,Low,4,6.75
3,2.0,Average,7,8.86
4,2.0,High,31,8.13
5,2.0,Low,1,9.00
6,3.0,Average,3,9.67
7,3.0,High,41,9.00
8,3.0,Low,2,10.00
9,4.0,Average,3,5.33


## 10. Age and Mental Health Analysis

### 10.1 Age Groups and Mental Health Outcomes
Let's examine how age relates to mental health outcomes among international students. We'll create age groups for better analysis.

In [14]:
# Create age groups and analyze mental health outcomes
query = """
SELECT 
    CASE 
        WHEN age < 20 THEN 'Under 20'
        WHEN age BETWEEN 20 AND 22 THEN '20-22'
        WHEN age BETWEEN 23 AND 25 THEN '23-25'
        WHEN age BETWEEN 26 AND 30 THEN '26-30'
        WHEN age > 30 THEN 'Over 30'
        ELSE 'Unknown'
    END AS age_group,
    COUNT(*) AS student_count,
    ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
    ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness,
    ROUND(CAST(AVG(toas) AS numeric), 2) AS avg_acculturative_stress
FROM students
WHERE inter_dom = 'Inter' AND age IS NOT NULL
GROUP BY age_group
ORDER BY MIN(age);
"""
run_query(query)

,age_group,student_count,avg_depression,avg_social_connectedness,avg_acculturative_stress
0,Under 20,72,8.40,36.40,76.49
1,20-22,87,8.36,37.99,73.69
2,23-25,27,7.37,36.07,79.56
3,26-30,11,5.73,40.64,72.82
4,Over 30,4,5.75,43.50,80.25


## 11. International vs Domestic Students Comparison

### 11.1 Mental Health Differences Between Student Types
Let's compare mental health outcomes between international and domestic students to understand the unique challenges faced by international students.

In [15]:
# Compare international vs domestic students
query = """
SELECT inter_dom, COUNT(*) AS student_count,
       ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
       ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness,
       ROUND(CAST(AVG(toas) AS numeric), 2) AS avg_acculturative_stress,
       ROUND(CAST(AVG(age) AS numeric), 1) AS avg_age
FROM students
WHERE inter_dom IS NOT NULL
GROUP BY inter_dom
ORDER BY inter_dom;
"""
run_query(query)

,inter_dom,student_count,avg_depression,avg_social_connectedness,avg_acculturative_stress,avg_age
0,Dom,67,8.61,37.64,62.84,20.4
1,Inter,201,8.04,37.42,75.56,21.0


## 12. Summary and Key Findings

### 12.1 Students at Highest Risk
Let's identify the group of international students who show the highest depression scores and lowest social connectedness to understand who might need the most support.

In [16]:
# Identify high-risk international students
query = """
SELECT 
    'High Depression (>15)' AS risk_category,
    COUNT(*) AS student_count,
    ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
    ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness
FROM students
WHERE inter_dom = 'Inter' AND todep > 15

UNION ALL

SELECT 
    'Low Social Connectedness (<30)' AS risk_category,
    COUNT(*) AS student_count,
    ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
    ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness
FROM students
WHERE inter_dom = 'Inter' AND tosc < 30

UNION ALL

SELECT 
    'High Acculturative Stress (>60)' AS risk_category,
    COUNT(*) AS student_count,
    ROUND(CAST(AVG(todep) AS numeric), 2) AS avg_depression,
    ROUND(CAST(AVG(tosc) AS numeric), 2) AS avg_social_connectedness
FROM students
WHERE inter_dom = 'Inter' AND toas > 60;
"""
run_query(query)

,risk_category,student_count,avg_depression,avg_social_connectedness
0,High Depression (>15),14,19.21,23.57
1,Low Social Connectedness (<30),39,12.49,22.51
2,High Acculturative Stress (>60),147,9.13,34.71
